In [1]:
import os

In [2]:
%pwd

'/home/izam/coding/Sentiment-Analysis/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/izam/coding/Sentiment-Analysis'

In [5]:
import pandas as pd

In [6]:
data = pd.read_csv("artifacts/data_ingestion/sentiment_data/train.txt", delimiter=';')
data.head()

,text,label
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18000 entries, 0 to 17999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    18000 non-null  object
 1   label   18000 non-null  object
dtypes: object(2)
memory usage: 281.4+ KB


In [8]:
data.isnull().sum()

text     0
label    0
dtype: int64

In [9]:
data.shape

(18000, 2)

In [10]:
list(data.columns)

['text', 'label']

In [11]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict
    delimeter: str

In [12]:
from sentimentAnalysis.constants import *
from sentimentAnalysis.utils.common import read_yaml, create_directories

In [13]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS
        delimeter = self.schema.DELIMITER

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema=schema,
            delimeter=delimeter
        )

        return data_validation_config

In [14]:
import os
from sentimentAnalysis import logger

In [15]:
class DataValiadtion:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir, delimiter=self.config.delimeter)
            all_cols = list(data.columns)
            all_schema = self.config.all_schema.keys()

            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e



In [16]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValiadtion(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2023-12-09 12:16:45,609: INFO: common: yaml file: config/config.yaml loaded successfully]
[2023-12-09 12:16:45,610: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-09 12:16:45,611: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-09 12:16:45,612: INFO: common: created directory at: artifacts]
[2023-12-09 12:16:45,613: INFO: common: created directory at: artifacts/data_validation]
